In [2]:
import csv
import json
import requests
from xml.etree import cElementTree as ET

# Read Zillow Web Services ID (ZWSID) from config file
config = json.loads(open('config.json').read())
zws_id = config['zws-id']

def getDeepSearchResults(address, zipcode):
    """"Get the deep search results including Zillow Id and house size using 
        Zillow GetDeepSearchResults API
    
    Args:
        address: house address get from SF housing inventory data
        zipcode: house zipcode calcualted from address using geocoding lib
    Returns:
        a tuple contains Zillow Id and house size if applicable
        
    """"
    deep_search_url = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm'
    params = {
        'address': address,
        'citystatezip': zipcode,
        'zws-id': zws_id
    }
    
    attr_mapping = {
        'zillow_id': 'result/zpid',
        'home_size': 'result/finishedSqFt',
    }
    
    r = requests.get(deep_search_url, params=params)
    data = ET.fromstring(r.text).findall('response/results')
    if not data:
        return ['', '']
    else:
        data = data[0]
    
    result = []
    for attr in attr_mapping.__iter__():
        try:
            result.append(data.find(attr_mapping[attr]).text)
        except AttributeError:
            result.append('')
    
    return result

def getRentEstimates(zillow_id):
    """"Get the rent zestimate from Zillow Id using Zillow GetZestimate API
    
    Args:
        zillow_id: Zillow Id for a house
    Returns:
        the rent zestimate if applicable
        
    """"
    rent_estimates_url = 'http://www.zillow.com/webservice/GetZestimate.htm'
    params = {
        'zpid': zillow_id,
        'zws-id': zws_id,
        'rentzestimate': 'true'
    }
    attr_mapping = {
        'rent_zestimate_amount': 'rentzestimate/amount',
    }
    
    r = requests.get(rent_estimates_url, params=params)
    data = ET.fromstring(r.text).findall('response')
    
    if not data:
        return ''
    else:
        data = data[0]
        
    for attr in attr_mapping.__iter__():
        try:
            return data.find(attr_mapping[attr]).text
        except AttributeError:
            return ''

with open('data/2012_Housing_Inventory_latlng_zip.csv', 'r') as csvfile:
    with open('2012_zillow.csv', 'w') as csv_out:
        writer = csv.writer(csv_out, lineterminator='\n')
        reader = csv.reader(csvfile)
        data = []
        
        # Append ZILLOW_ID, RENT_ESTIMATE and HOUSE_SIZE column to CSV file
        header = reader.next()
        header.append("ZILLOW_ID")
        header.append("RENT_ESTIMATE")
        header.append("HOUSE_SIZE")
        data.append(header)
        
        # Look the address and zipcode to find Zillow Id and house size
        for row in reader:
            # address = row[7] + " San Francisco, CA"
            address = row[0]
            zipcode = row[17]
            
            house_details = getDeepSearchResults(address, zipcode)
            zpid = house_details[0]
            rent_estimate = getRentEstimates(zpid)
            house_size = house_details[1]
            
            row.append(zpid)
            row.append(rent_estimate)
            row.append(house_size)
            data.append(row)
            
        writer.writerows(data)

4290
